In [42]:
# Import os
import requests
import openai
from dotenv import load_dotenv


In [43]:
def fetch_movies_from_tmdb(mood, decade, min_rating, country):
    """Fetch movies from TMDB based on mood, decade, minimum rating, and country of origin."""
    genres = get_genres_for_mood(mood)
    genre_str = ','.join(genres)
    start_date = f'{decade}-01-01'
    end_date = f'{decade+9}-12-31'
    load_dotenv()
    api_key = os.getenv('TMDB_API_KEY')
    url = "https://api.themoviedb.org/3/discover/movie"
    params = {
        "api_key": api_key,
        "with_genres": genre_str,
        "sort_by": "popularity.desc",
        "primary_release_date.gte": start_date,
        "primary_release_date.lte": end_date,
        "vote_average.gte": min_rating,
        "region": country.upper(),
    }
    response = requests.get(url, params=params)
    return response.json().get("results", [])

In [44]:
# Load OpenAI API key from .env file (add OPENAI_API_KEY=your_key to .env)
load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')

def generate_movie_recommendations(mood, movies, n=3):
    """Use OpenAI GPT to recommend and describe n movies for the given mood, showing TMDB rating next to each film name."""
    if not openai_api_key:
        raise ValueError('OpenAI API key not found. Please add OPENAI_API_KEY to your .env file.')
    openai.api_key = openai_api_key
    movie_list = '\n'.join([f"- {m['title']} (TMDB rating: {m.get('vote_average', 'N/A')}): {m.get('overview', 'No description available.')}" for m in movies[:10]])
    prompt = (f"User mood: {mood}\n"
              f"Here are some movies fetched from TMDB that might fit this mood (with their TMDB ratings):\n{movie_list}\n\n"
              f"Please recommend and describe {n} movies from this list that best fit the user's mood. For each recommended movie, you must display the TMDB rating next to the movie title in your output. Respond in a friendly, engaging way.")
    response = openai.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": prompt}],
        max_tokens=500,
        temperature=0.7,
    )
    return response.choices[0].message.content

In [48]:
# Example usage: fetch movies and generate recommendations using the function
mood = 'happy'  # e.g., 'sad', 'excited', etc.
decade = 2000  # e.g., 1980, 1990, 2000, 2010, 2020
min_rating = 8.0  # Minimum average vote
country = 'US'    # 2-letter country code

movies = fetch_movies_from_tmdb(mood, decade, min_rating, country)

In [49]:
# Generate and display AI-powered recommendations
output = generate_movie_recommendations(mood, movies, n=10)
print(output)

I'm so glad to hear that you're feeling happy! Here are 10 movies from the list that might match your cheerful mood:

1. Onni von Sopanen (TMDB rating: 9.0) - Join 11-year-old Onni on his journey to uncover the truth about his parents in this heartwarming tale.
2. Adventure Time (TMDB rating: 8.2) - Dive into the whimsical world of Pen and Jake as they embark on a rescue mission to save Princess Bubblegum.
3. The 1st 13th Annual Fancy Anvil Awards Show Program Special: Live in Stereo (TMDB rating: 10.0) - Get ready for a fun-filled awards show celebrating cartoon excellence with Cartoon Network!
4. Codename: Kids Next Door: Operation I.N.T.E.R.V.I.E.W.S. (TMDB rating: 8.276) - Join the former KND operatives as they reminisce about their last mission in a thrilling scavenger hunt adventure.
5. Lotte from Gadgetville (TMDB rating: 8.6) - Experience a village full of inventors and join Lotte on an exciting journey with new friends from distant lands.
6. Little Einsteins: Our Big Huge Adve